# Практическая работа номер 2
## Калибровка камеры
В этой практической работе вам предстоит освоить процесс калиборвки камеры от детектирования углов шахматки до запуска оптимизационной процедуры, вычисляющей параметры камеры


для начала проверим что все модули установлены

In [ ]:
from __future__ import print_function
import numpy as np
import scipy
import scipy.optimize
import matplotlib
import matplotlib.pyplot as plt
import cv2

import sys

print('numpy version is', np.__version__)
print('opnecv version is', cv2.__version__)
print('scipy version is', scipy.__version__)
print('python version is', sys.version)
print('matplotlib version is', matplotlib.__version__)

### Открыть видео
Для зачитывания кадров видео будем использовать opencv
Чтобы открыть видео или DirectShow\v4l камеру нужно воспользоваться классм cv2.VideoCapture

Пример зачитывания:

In [ ]:
cap = cv2.VideoCapture('../data/calib_video/2017_11_07.mp4')
print('video is', '' if cap.isOpened() else 'not', 'opened')

%matplotlib inline
res, img = cap.read()
plt.imshow(img)

## Детектирование углов шахматки

Исходными данными для задачи калибровки являются точки на объекте, расположение которых нам заведомо известно.
Например - углы шахматной доски. Калибровочный паттерн может быть и другим. 

Чтобы задетектировать уголки шахматки можно воспользоваться детекторами особых точек, например FAST:

In [ ]:
fast = cv2.FastFeatureDetector_create()
kp = fast.detect(img,None)
img2 = cv2.drawKeypoints(img, kp, None, color=(255,0,0))
plt.imshow(img2)

Более удобным способом, сделанным специально для калибровки, является функция findChessboardCorners. Помимо детектирования шахматки, она перенумировывет определенным образом углы. Это важно, ведь если их нумерация перепутается, оптимизируемая функция будет содержать ошибку

In [ ]:
flags = cv2.CALIB_CB_ADAPTIVE_THRESH + cv2.CALIB_CB_NORMALIZE_IMAGE + cv2.CALIB_CB_FAST_CHECK   
res, corners = cv2.findChessboardCorners(img, (7,7), flags=flags)

img2 = cv2.drawChessboardCorners(img.copy(), (7,7), corners, res)
plt.imshow(img2)

### Задание 2.1 (*)
Напишите функцию, детектирующую и пронумеровывающую углы шахматки, аналогично cv2.findChessboardCorners, но на основе анализа результатов FAST-детектора.
Сравните время их работы с помощью модуля timeit

In [ ]:
# решение задания 2.1 (*)

Теперь нужно запрограммировать модель объекта. координаты (X, Y, Z) точек шахматной доски в с.к., связанной с доской.

In [ ]:
import itertools

def get_obj_points(chessboard_size, cell_size):
    pts = []
    pts.append([(x * cell_size, y * cell_size, 0)
                for (y, x) in itertools.product(range(0, chessboard_size[1]),
                                                range(0, chessboard_size[0]))])
    return np.array(pts, dtype='float32').reshape((
        chessboard_size[0] * chessboard_size[1], 3, 1))


obj_points = get_obj_points((7,7), 0.1) # шахматная доска 7 на 7, размер клетки - 10см.

### Задание 2.2
1) Необходимо ответить на вопрос, как расположена с.к. паттерна-шахматки? 

2) Отобразите с помощью функции drawChessboardCorners, как пронумерованы клетки шахматки в object_points 

#### Ответ на 2.2.1)
...

In [ ]:
# код ответа на 2.2.2)


## Начальные значения

Любая численная оптимизация работает лучше, если начинать вблизи экстремума. 
В задаче калибровки нормальным будет взять за начальное приближение нулевой вектор дисторсии и фокусное расстояние, прикинутое из углов раствора камеры.
Сделаем это.

### Задание 2.3
Написать функцию, генерирующую матрицу камеры и параметры дисторсии для заданного размера изображения и угла раствора камеры
Функция принимает параметры:
* angle_deg --- горизонтальный угол раствора камеры в градусах
* img_size --- размеры изображения

In [ ]:
# решение задачи 2.3
def guess_intrinsics(angle_deg, img_size):
    pass;

In [ ]:
p_init = np.eye(3, dtype=np.float32)
d_init = np.zeros([3,], dtype=np.float32)
r_init = np.zeros([3,], dtype=np.float32)
t_init = np.array([0, 0, 3.0], dtype=np.float32)

# воспользуемся нашим начальным приближением
p_init, d_init = guess_intrinsics(90, img.shape[1::-1])

# p_init = np.array([[965.59562, 0.0, 1024.0],
#                    [0.0, 965.59562, 544.0],
#                    [0.0, 0.0, 1]],
#                   dtype='float32')

# полный вектор начальных значений.
x_init = np.array([p_init[0,0], p_init[0,2], p_init[1,2],
                   d_init[0], d_init[1], d_init[2],
                   r_init[0], r_init[1], r_init[2], 
                   t_init[0], t_init[1], t_init[2]], dtype=np.float32)


## Минимизируемая функция
Суть калибровки - в минимизации ошибки репроекции точек шахматки по набору неизвестных параметров.
Для этой минимизации мы будем использовать готовую реализацию алгоритма Левенберга-Марквадта из пакета научных вычиследний scipy: ```scipy.optimize.least_squares```
https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.least_squares.html#scipy.optimize.least_squares

In [ ]:
# вариант 1. используем Opencv-шную функцию.
def projection_helper(x):
    cam_mat = np.array([[x[0], 0, x[1]],
                        [0, x[0], x[2]],
                        [0,    0,    1]],
                      dtype=np.float32)
    dist = np.array([x[3], x[4], 0, 0, x[5]], dtype=np.float32)
    r = x[6:9]
    t = x[9:]
    # функция cv2.projectPoints сразу возвращает и якобиан тоже
    pts, jac = cv2.projectPoints(obj_points, r, t, cam_mat, dist)
    return (pts, jac)

def f_projection(x):
    pts, jac = projection_helper(x)
    pts = pts - corners
    #return pts.reshape(-1)
    return np.sqrt(pts[:,0,0] ** 2 + pts[:,0,1] ** 2)

def jac_projection(x):
    pts, jac = projection_helper(x)
    pts = pts - corners
    dists = np.sqrt(pts[:,0,0] ** 2 + pts[:,0,1] ** 2)
    jac = jac.reshape(pts.shape[0], 2, -1)
    # составляем новый якобиан из наших переменных
    new_jac = jac[:, :, [6, 8, 9, 10, 11, 14, 0, 1, 2, 3, 4, 5]]
    new_jac[:, :, 0] += jac[:, :, 7]
    jac = new_jac
    # вычисляем якобиан целевой функции
    #return jac.reshape(-1, jac.shape[-1])
    return (pts[:, 0, 0] / dists)[:, np.newaxis] * jac[:, 0, :] + \
           (pts[:, 0, 1] / dists)[:, np.newaxis] * jac[:, 1, :]

### Задание 2.4
Отобразите текущее значение проекции точек на картинке img

In [ ]:
# решение задачи 2.4




## Запуск минимизации
Итак, пришло время запустить оптимизацию и получить откалиброванные значения
функция ```scipy.optimize.least_squares``` имеет множество параметров, таких как критерии останова, масштабы аргуметов и целевой функции, кастомизация функции потерь и прочее.
Нас интересуют только `method`, `fun` и `jac` и `x0`

In [ ]:
opt_res = scipy.optimize.least_squares(f_projection, x_init, jac=jac_projection, method='lm')
print(opt_res)

In [ ]:
x = opt_res['x']
cam_mat = np.array([[x[0], 0, x[1]],
                    [0, x[0], x[2]],
                    [0,    0,    1]],
                      dtype=np.float32)
dist = np.array([x[3], x[4], 0, 0, x[5]], dtype=np.float32)
r = x[6:9]
t = x[9:]
print('K = ', cam_mat)
print('distortion = ', dist)
print('r = ', r)
print('t = ', t)

### Задание 2.5 
Реализуйте вычисление f через функцию projectpt из первой практической работы. В этом случае вычислять якобиан будем с помощью разностной схемы. Запустите минимизацию с вашей функцией и соответствующим параметром jac

In [ ]:
# решение задачи 2.5


### Задание 2.6 (*)
Реализуйте вычисление Якобиана для вашей функции projectPt. Запустите оптимизацию. Вывод якобиана можете в tex-нотации набить в отдельной клетке markdown, вот так: $\frac {\partial y} {\partial \theta}$

## Проверка решения
Попробуем воспользоваться полученными значениями

In [ ]:
und = cv2.undistort(img, cam_mat, dist)
pts, _ = cv2.projectPoints(obj_points, r, t, cam_mat, np.zeros((5,), dtype=np.float32))
und = cv2.drawChessboardCorners(und, (7,7), pts, res)
plt.imshow(und)

Как видно, целевая функция, действитео, оптимизирована: точки лежат довольно хорошо на искаженной картинке. Однако мы явно не добились желаемого эффекта.

Проблем в том, что наш паттерн находился только в одном регионе изображения, и алгоритм оптимизации пришел в минимум, который не будет удовлетворять другим ионам изображения. В машинном обучении это называется переобучение или overfitting.

Для того чтобы получить правильные параметры калибровки, нужно испьзовать больше чем один кадр. Так, чтобы изображение паттерна в равной степени заметало все регионы исходного изображения.

### Задание 2.7
Модифицируйте целевую функцию так, чтобы она брала в рассмотрение Несколько кдров одновременно. (попробуйте взять в районе 40).
Запустите алгоритм на 

1) всех изображениях сразу

2) рандомного подмножества изображений (*)


Note: при решении этого задания следует переписать все что было ранее в клетках ниже, а не изменять то что сверху уже написано и выполнилось, чтобы упростить преподавателю проверку вашего задания.

In [ ]:
# решение задания 2.7

## Конец
надеюсь, задание вам понравилось и делать его было так же интересно, как мне было интересно его составлять